In [1]:
from safetensors.torch import load_file
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from sagemaker.pytorch import PyTorch

2025-02-20 18:38:18.953625: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [13]:
import unsloth

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

## Loading the Finetuned Model
SAVED_MODEL is the lora huggingface repo of the model that has been fine-tuned. INFER_MODEL is the huggingface repo of the model that has the lora weights merged with the base weights for inferring.

In [11]:
SAVED_MODEL = "Alexis-Az/Story-Generation-LlaMA-3.1-8B-10k"
INFER_MODEL= "Alexis-Az/Story-Generation-Model"
max_seq_length = 1024

In [12]:
adapter_model, tokenizer = unsloth.FastLanguageModel.from_pretrained(SAVED_MODEL, load_in_4bit=True)

NameError: name 'unsloth' is not defined

In [9]:
adapter_model.save_pretrained_merged("Story-Generation-LlaMA-3.1-8B-10k", tokenizer)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 15.61 out of 30.89 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 41%|████      | 13/32 [00:01<00:01, 14.00it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:14<00:00,  2.18it/s]


Unsloth: Saving tokenizer... Done.
Done.


In [ ]:
adapter_model

In [9]:
!ls

'Finetuning Story Generation Model.ipynb'   hf_auth.ipynb
 README.md				    inference_container
 Story-Generation-LlaMA-3.1-8B-10k	    load_data.ipynb
 feature_engineering.ipynb		    unsloth_compiled_cache
 format_pth_model.ipynb


## Saving the Merged Model 
The model will be pushed to the INFER_MODEL huggingface repo, and will also be saved as a model .pth file for use in the container for inferrence.

In [2]:
pth_model = AutoModelForCausalLM.from_pretrained("./Story-Generation-LlaMA-3.1-8B-10k")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("./Story-Generation-LlaMA-3.1-8B-10k")

In [6]:
pth_model.push_to_hub(
    INFER_MODEL,
    tokenizer=tokenizer,
    safe_serialization=True,
    create_pr=True,
    max_shard_size="3GB",
)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Alexis-Az/Story-Generation-Model/commit/785a0842eca3e73c6ff7594dc7d1574f78353ce3', commit_message='Upload LlamaForCausalLM', commit_description='', oid='785a0842eca3e73c6ff7594dc7d1574f78353ce3', pr_url='https://huggingface.co/Alexis-Az/Story-Generation-Model/discussions/3', repo_url=RepoUrl('https://huggingface.co/Alexis-Az/Story-Generation-Model', endpoint='https://huggingface.co', repo_type='model', repo_id='Alexis-Az/Story-Generation-Model'), pr_revision='refs/pr/3', pr_num=3)

In [ ]:
from pathlib import Path
model_dir = "./working_dir/models/serve"
!mkdir -p {model_dir}

In [3]:
torch.save(pth_model.state_dict(), model_dir + '/model.pth')

In [11]:
# save the model directory as a tar.gz
import subprocess

In [5]:
!ls

'Finetuning Story Generation Model.ipynb'   inference_container
 README.md				    load_data.ipynb
 deploy_model.ipynb			    sample_input.txt
 feature_engineering.ipynb		    unsloth_compiled_cache
 format_pth_model.ipynb			    working_dir
 hf_auth.ipynb


In [12]:
source_directory = "./working_dir/models/serve"  # Replace with the actual directory path
output_file = "serve"

In [ ]:
subprocess.check_call(("tar", '-use-compress-program="pigz --best --recursive"', "-cf", output_file, source_directory))